In [1]:
import pandas as pd
import billboard as bd #to access this data: https://www.billboard.com/charts/r-b-hip-hop-songs
import pickle

In [70]:
def create_chart(chart, date):
    '''Produces chart for given date and type in pandas dataframe form with all attributes'''
   # hot100 = bd.ChartData(chart_type, date)
    artists = [x.artist for x in chart]
    titles = [x.title for x in chart]
    peak = [x.peakPos for x in chart]
    last = [x.lastPos for x in chart]
    weeks = [x.weeks for x in chart]
    rank = [x.rank for x in chart]
    isNew = [x.isNew for x in chart]
    billB100 = pd.DataFrame({'Date': date,'Artist':artists, 'Title': titles, 'Peak_Pos': peak, 
                             'Last_Week_Pos': last, 'Rank': rank, 'isNew': isNew })
    return billB100

In [84]:
root_chart= bd.ChartData('r-b-hip-hop-songs', '2019-05-11')

counter = 0
chartList = []
while root_chart.previousDate:
        chartList.append(create_chart(root_chart, root_chart.date))
        root_chart = bd.ChartData('r-b-hip-hop-songs', root_chart.previousDate)
        counter += 1
        if(counter == 520): #get last 52 weeks * #years
            break
full = pd.concat(chartList)

In [117]:
fullDF = full.reset_index()
fullDF.to_pickle("R&B10yrDF.pkl")

In [118]:
fullDF

,index,Artist,Date,Last_Week_Pos,Peak_Pos,Rank,Title,isNew
0,0,Lil Nas X Featuring Billy Ray Cyrus,2019-05-11,1,1,1,Old Town Road,False
1,1,Post Malone,2019-05-11,2,1,2,Wow.,False
2,2,Post Malone & Swae Lee,2019-05-11,3,1,3,Sunflower (Spider-Man: Into The Spider-Verse),False
3,3,Khalid,2019-05-11,4,4,4,Talk,False
4,4,J. Cole,2019-05-11,5,2,5,Middle Child,False
5,5,Meek Mill Featuring Drake,2019-05-11,7,2,6,Going Bad,False
6,6,Cardi B & Bruno Mars,2019-05-11,6,1,7,Please Me,False
7,7,Khalid,2019-05-11,8,5,8,Better,False
8,8,21 Savage,2019-05-11,11,5,9,A Lot,False
9,9,Mustard & Migos,2019-05-11,10,10,10,Pure Water,False


In [119]:
#now can read in data from here
fullDF = pd.read_pickle("R&B10yrDF.pkl")
fullDF.Date.value_counts() #you can see here that each  week only has 50 songs - please help me find out why :)b

2016-09-24    50
2015-05-16    50
2011-06-25    50
2018-12-08    50
2015-09-12    50
2012-03-10    50
2011-02-26    50
2017-12-02    50
2018-08-04    50
2015-10-31    50
2013-12-28    50
2018-06-30    50
2017-08-05    50
2013-02-02    50
2013-03-23    50
2017-11-11    50
2010-05-01    50
2018-01-27    50
2009-10-03    50
2017-09-09    50
2011-09-03    50
2017-03-11    50
2018-03-10    50
2011-05-07    50
2017-09-23    50
2009-08-29    50
2013-02-16    50
2009-07-11    50
2012-05-12    50
2019-01-19    50
              ..
2017-05-20    50
2014-03-01    50
2012-02-18    50
2010-04-10    50
2009-12-26    50
2017-08-12    50
2011-11-19    50
2015-06-27    50
2016-01-16    50
2019-03-16    50
2013-01-05    50
2014-05-03    50
2012-03-17    50
2015-11-28    50
2015-01-24    50
2017-10-28    50
2014-09-06    50
2015-04-18    50
2016-03-19    50
2019-05-11    50
2015-08-22    50
2010-09-18    50
2011-09-24    50
2014-08-02    50
2011-03-12    50
2013-01-26    50
2016-05-28    50
2011-12-31    

In [120]:
!dir

 Volume in drive C is Windows
 Volume Serial Number is 32A6-5CEF

 Directory of C:\Users\gr8\Documents\STA141C\Project

05/11/2019  08:41 PM    <DIR>          .
05/11/2019  08:41 PM    <DIR>          ..
05/11/2019  05:11 PM    <DIR>          .ipynb_checkpoints
05/07/2019  01:50 PM    <DIR>          CollegeScorecard_Raw_Data
04/30/2019  01:32 PM         1,102,815 communities.data
04/30/2019  01:32 PM            27,179 communities.names
05/11/2019  08:36 PM            38,463 R
05/11/2019  08:42 PM         1,996,613 R&B10yrDF.pkl
05/11/2019  05:19 PM    <DIR>          STA141C_Project
               4 File(s)      3,165,070 bytes
               5 Dir(s)  548,266,471,424 bytes free
